In [1]:
%pylab --no-import-all inline

Populating the interactive namespace from numpy and matplotlib


# Load and preprocess 1976 data

Time to start looking at other years!

---

In [14]:
import pandas as pd

## Load the data.

---

If this fails, be sure that you've saved your own data in the prescribed location, then retry.

In [15]:
file = "../data/interim/1976data.dta"  
# Matt Wilson converted the older Stata file to the one we use.
df_rawest = pd.read_stata(file)

In [16]:
good_columns = [
    # Demographic
    'V763174',  # SUMMARY-R'S PARTY ID
    
    'V763796',  # OPIN:WHEN ALLOW ABORTION (1: never)

    'V763273',  # Private vs public insurance
    'V763241',  # GOVT GUAR JOB/S.L  (1: guarantee)
    'V763353',  # Gov't should spend less, even if cutting health and education.

    'V763264',  # MNRTY GRP AID SCL (1: help)
    'V763757',  # THE POOR ARE POOR BECAUSE THE AMERICAN WAY OF LIFE DOESN'T GIVE ALL PEOPLE AN EQUAL CHANCE? (1: agree)
]
df_raw = df_rawest[good_columns]

In [37]:
def convert_to_int(s):
    """Turn ANES data entry into an integer.
    
    >>> convert_to_int("1. Govt should provide many fewer services")
    1
    >>> convert_to_int("2")
    2
    """
    try:
        return int(s.partition('.')[0])
    except ValueError:
        warnings.warn("Couldn't convert: "+s)
        return np.nan
    except AttributeError:
        return s
    
def not_informative_to_nan(x):
    """Convert non-informative values to missing.
    
    ANES codes various non-answers as 8, 9, and 0.
    For instance, if a question does not pertain to the 
    respondent.
    """
    return np.nan if x in {8, 9, 0} else x


def fix_V763168(x):
    return x if x in {1, 2} else 3 if x == 5 else np.nan

non_pid_columns = list(df.columns)
non_pid_columns.remove('V763174')

df = df_raw.applymap(convert_to_int)
df[non_pid_columns] = df[non_pid_columns].applymap(not_informative_to_nan)  # Dropped because its info is different.

df.loc[:, 'V763796'] = df.V763796.apply(lambda x: np.nan if x in {7, 8, 9, 0} else -x)

# Code so that liberal is higher numbers
df.loc[:, 'V763174'] = df.V763174.apply(lambda x: np.nan if x >= 7 else -x)  # 7: other minor party, 8: apolitical, 9: NA

# df.loc[:, 'V880951'] = df.V880951.apply(lambda x: -x)  # 1: moral relativism, 5: no relativism
# df.loc[:, 'V880952'] = df.V880952.apply(lambda x: -x)  # Tolerance. 1: tolerance, 7: not
# df.loc[:, 'V880852'] = df.V880852.apply(lambda x: -x)  # protect gays against discrimination

df.loc[:, 'V763273'] = df.V763273.apply(lambda x: -x)  # Gov't insurance?
df.loc[:, 'V763241'] = df.V763241.apply(lambda x: -x)  # Gov't guaranteed job?

df.loc[:, 'V763264'] = df.V763264.apply(lambda x: -x)  # affirmative action
df.loc[:, 'V763757'] = df.V763757.apply(lambda x: -x)  # Poor?
# df.loc[:, 'V880964'] = df.V880964.apply(lambda x: -x)  # Systemic factors?

df.rename(inplace=True, columns=dict(zip(
    good_columns,
    ["PartyID",
    
    "Abortion",
#     "MoralRelativism",
#     "NewerLifestyles",
#     "MoralTolerance",
#     "TraditionalFamilies",
#     "GayJobDiscrimination",
#     "GayMilitaryService",

    "NationalHealthInsurance",
    "StandardOfLiving",
    "ServicesVsSpending",

    "AffirmativeAction",
#     "RacialResentment1",
#     "RacialResentment2",
#     "RacialResentment3",
#     "RacialResentment4",
     "Poverty",
    ]
)))

In [38]:
df.tail()

,PartyID,Abortion,NationalHealthInsurance,StandardOfLiving,ServicesVsSpending,AffirmativeAction,Poverty
2243,-5.0,NaN,NaN,NaN,5.0,-3.0,NaN
2244,-5.0,NaN,-3.0,-7.0,5.0,-5.0,NaN
2245,-5.0,-1.0,-2.0,-7.0,5.0,-6.0,-4.0
2246,0.0,-4.0,-1.0,-4.0,5.0,-7.0,-5.0
2247,-3.0,-4.0,-7.0,-7.0,5.0,-6.0,-5.0


In [39]:
print("Variables now available: df")

Variables now available: df


In [40]:
df_rawest.V763174.value_counts()

1. WEAK DEM 5 5 0      547
0. STRONG DEM 5 1 0    338
5. WEAK REP 1 5 0      327
3. IND-IND 2 0 3       315
2. IND-DEM 2 0 5       260
4. IND-REP 2 0 1       218
6. STRONG REP 1 1 0    208
8. APOLITICAL 3 0 3     20
7. OTH,MINOR PARTY,      9
9. NA,DK 8 0 0           6
Name: V763174, dtype: int64

In [41]:
df.PartyID.value_counts()

-1.0    547
 0.0    338
-5.0    327
-3.0    315
-2.0    260
-4.0    218
-6.0    208
Name: PartyID, dtype: int64

In [42]:
df.to_csv("../data/processed/1976.csv")